In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.simplefilter('ignore')

In [2]:
df = pd.read_csv("insurance.csv")
df.rename(columns={"charges":"expenses"},inplace = True)
df

,age,sex,bmi,children,smoker,region,expenses
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [4]:
continuos_var = ['age','bmi','expenses']
discrete_var = ['sex','smoker','region']
count_var = ['children']

In [5]:
df[continuos_var].describe()

,age,bmi,expenses
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,13270.422265
std,14.049960,6.098187,12110.011237
min,18.000000,15.960000,1121.873900
25%,27.000000,26.296250,4740.287150
50%,39.000000,30.400000,9382.033000
75%,51.000000,34.693750,16639.912515
max,64.000000,53.130000,63770.428010


In [6]:
df[discrete_var].describe()

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,male,no,southeast
freq,676,1064,364


In [7]:
df[continuos_var].corr()

,age,bmi,expenses
age,1.000000,0.109272,0.299008
bmi,0.109272,1.000000,0.198341
expenses,0.299008,0.198341,1.000000


In [8]:
df.drop_duplicates(inplace=True)
df.drop('region',axis = 1,inplace = True) # vaadu ekkada unte enti?

In [9]:
df['sex'].replace({'female':0,'male':1},inplace = True)
df['smoker'].replace({'no':0,'yes':1},inplace = True)

In [10]:
x = df.drop('expenses',axis = 1)
y = df['expenses']
df

,age,sex,bmi,children,smoker,expenses
0,19,0,27.900,0,1,16884.92400
1,18,1,33.770,1,0,1725.55230
2,28,1,33.000,3,0,4449.46200
3,33,1,22.705,0,0,21984.47061
4,32,1,28.880,0,0,3866.85520
...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10600.54830
1334,18,0,31.920,0,0,2205.98080
1335,18,0,36.850,0,0,1629.83350
1336,21,0,25.800,0,0,2007.94500


In [11]:
train_r2 = []
cv = []
test_r2 = []

for i in range(0,100):
    
    from sklearn.model_selection import train_test_split
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=i)
    
    from sklearn.linear_model import Lasso
    model=Lasso()
    model.fit(x_train,y_train)

    ypred_train = model.predict(x_train)
    ypred_test = model.predict(x_test)
    
    from sklearn.model_selection import cross_val_score
    cv.append(cross_val_score(model,x_train,y_train,cv=5).mean())
    
    from sklearn.metrics import r2_score
    train_r2.append(r2_score(y_train,ypred_train))
    test_r2.append(r2_score(y_test,ypred_test))
    
em =pd.DataFrame({"train_r2":train_r2,"cv":cv})

# for best random state number...just check train == cv or not

gm = em[(abs(em["train_r2"]-em["cv"])<=0.05)] 

# Difference b/w train and cv must be less than 0.05 itself

rs=gm[gm["cv"]==gm["cv"].max()].index.tolist()[0]

rs  #[it should be 9]

55

# Lasso stands for Least Absolute Shrinkage and Selection Operator(LASSO)

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state = 9)


In [13]:
from sklearn.model_selection import GridSearchCV
estimator = Lasso()
param_grid = {'alpha':list(range(0,70))}
modelhp = GridSearchCV(estimator,param_grid,cv=5,scoring='r2')
modelhp.fit(x_train,y_train)
modelhp.best_params_

{'alpha': 60}

In [14]:
from sklearn.linear_model import Lasso
model = Lasso(alpha = 60)
model.fit(x_train,y_train)

print("intercep values : ",model.intercept_)
print("coefficient values : ",model.coef_)

ypred_train = model.predict (x_train)
ypred_test = model.predict(x_test)

from sklearn.model_selection import cross_val_score
print("cv_score :",cross_val_score(model,x_train,y_train,cv=5).mean())

from sklearn.metrics import r2_score
print("train_r2 :", r2_score(y_train,ypred_train))
print("test_r2 :", r2_score(y_test,ypred_test))

intercep values :  -12040.593571887146
coefficient values :  [  264.38161528    -0.           316.89561192   373.0709729
 23622.96074097]
cv_score : 0.7537058055900724
train_r2 : 0.7591801991925938
test_r2 : 0.7009201105342885


# **Second column slope is zero means remove that column(sex)**

In [15]:
x = x.drop('sex',axis = 1)
#df.drop('sex',axis = 1,inplace = True)
y = df['expenses']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state = 9)

from sklearn.linear_model import Lasso
model1 = Lasso(alpha = 60)
model1.fit(x_train,y_train)

print("intercep values : ",model1.intercept_)
print("coefficient values : ",model1.coef_)

ypred_train = model1.predict (x_train)
ypred_test = model1.predict(x_test)

from sklearn.model_selection import cross_val_score
print("cv_score :",cross_val_score(model1,x_train,y_train,cv=5).mean())

from sklearn.metrics import r2_score
print("train_r2 :", r2_score(y_train,ypred_train))
print("test_r2 :", r2_score(y_test,ypred_test))

intercep values :  -12040.588905108545
coefficient values :  [  264.38141961   316.89570704   373.07104773 23622.96061177]
cv_score : 0.753814164737532
train_r2 : 0.7591801990707372
test_r2 : 0.7009201316340361


In [16]:
input_data = {'age':35,
             'sex':'male',
             'bmi':31.4,
             'children':5,
             'smoker':'yes',
             'region':'southeast'}
df_test =pd.DataFrame(input_data,index=[0])
df_test

,age,sex,bmi,children,smoker,region
0,35,male,31.4,5,yes,southeast


In [17]:
df_test.drop('region',axis = 1,inplace = True)
df_test['sex'].replace({'female':0,'male':1},inplace = True)
df_test['smoker'].replace({'no':0,'yes':1},inplace = True)
df_test.drop('sex',axis = 1,inplace = True)
df_test

,age,bmi,children,smoker
0,35,31.4,5,1


In [18]:
model1.predict(df_test)

array([32651.60183275])

# IT'S **RIDGE** TIME

In [19]:
train_r2 = []
cv = []
test_r2 = []

for i in range(0,100):
    
    from sklearn.model_selection import train_test_split
    x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=i)
    
    from sklearn.linear_model import Ridge
    model=Ridge()
    model.fit(x_train,y_train)

    ypred_train = model.predict(x_train)
    ypred_test = model.predict(x_test)
    
    from sklearn.model_selection import cross_val_score
    cv.append(cross_val_score(model,x_train,y_train,cv=5).mean())
    
    from sklearn.metrics import r2_score
    train_r2.append(r2_score(y_train,ypred_train))
    test_r2.append(r2_score(y_test,ypred_test))
    
em =pd.DataFrame({"train_r2":train_r2,"cv":cv})

# for best random state number...just check train == cv or not

gm = em[(abs(em["train_r2"]-em["cv"])<=0.05)] 

# Difference b/w train and cv must be less than 0.05 itself

rs=gm[gm["cv"]==gm["cv"].max()].index.tolist()[0]

rs  #[it should be 9]

55

In [21]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state = 9)

from sklearn.model_selection import GridSearchCV
estimator = Ridge()
param_grid = {'alpha':list(range(0,70))}
modelhp = GridSearchCV(estimator,param_grid,cv=5,scoring='r2')
modelhp.fit(x_train,y_train)
modelhp.best_params_

{'alpha': 1}

In [22]:
from sklearn.linear_model import Lasso
model = Lasso(alpha = 60)
model.fit(x_train,y_train)

print("intercep values : ",model.intercept_)
print("coefficient values : ",model.coef_)

ypred_train = model.predict (x_train)
ypred_test = model.predict(x_test)

from sklearn.model_selection import cross_val_score
print("cv_score :",cross_val_score(model,x_train,y_train,cv=5).mean())

from sklearn.metrics import r2_score
print("train_r2 :", r2_score(y_train,ypred_train))
print("test_r2 :", r2_score(y_test,ypred_test))

intercep values :  -12040.588905108545
coefficient values :  [  264.38141961   316.89570704   373.07104773 23622.96061177]
cv_score : 0.753814164737532
train_r2 : 0.7591801990707372
test_r2 : 0.7009201316340361


In [23]:
x = x.drop('sex',axis = 1)
#df.drop('sex',axis = 1,inplace = True)
y = df['expenses']

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state = 9)

from sklearn.linear_model import Lasso
model1 = Lasso(alpha = 60)
model1.fit(x_train,y_train)

print("intercep values : ",model1.intercept_)
print("coefficient values : ",model1.coef_)

ypred_train = model1.predict (x_train)
ypred_test = model1.predict(x_test)

from sklearn.model_selection import cross_val_score
print("cv_score :",cross_val_score(model1,x_train,y_train,cv=5).mean())

from sklearn.metrics import r2_score
print("train_r2 :", r2_score(y_train,ypred_train))
print("test_r2 :", r2_score(y_test,ypred_test))

KeyError: "['sex'] not found in axis"

In [ ]:
input_data = {'age':35,
             'sex':'male',
             'bmi':31.4,
             'children':5,
             'smoker':'yes',
             'region':'southeast'}
df_test =pd.DataFrame(input_data,index=[0])
df_test

In [ ]:
df_test.drop('region',axis = 1,inplace = True)
df_test['sex'].replace({'female':0,'male':1},inplace = True)
df_test['smoker'].replace({'no':0,'yes':1},inplace = True)
df_test.drop('sex',axis = 1,inplace = True)
df_test

In [ ]:
model1.predict(df_test)